In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
pwd

'/home/cloudcraftz/End-To-End-Wine-Quality'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig():
    root_dir: Path
    model_name: str
    train_data_path: Path
    test_data_path: Path
    model_path: Path
    evaluation_save_dir: Path
    target_column: str

In [5]:
from WineQuality.constants import *
from WineQuality.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH,schema_filepath = SCHEMA_FILE_PATH):
        
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_configs(self) -> list[ModelEvaluationConfig]:

        config = self.config.model_evaluation
        params = self.params
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_configs =[]

        for model_name, params_grid in params.items():
            model_evaluation_config = ModelEvaluationConfig(
                root_dir= config.root_dir,
                model_name= model_name,
                train_data_path= config.train_data_path,
                test_data_path= config.test_data_path,
                model_path= os.path.join(config.model_path, f"{model_name}/{model_name}.pkl"),
                target_column= schema.name,
                evaluation_save_dir= config.evaluation_save_dir

            )

            model_evaluation_configs.append(model_evaluation_config)

        return model_evaluation_configs



In [18]:
ConfigurationManager().get_model_evaluation_configs()

[2025-03-22 22:39:15,996: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-22 22:39:15,998: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-22 22:39:15,999: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-22 22:39:16,000: INFO: common: created directory at: artifacts]
[2025-03-22 22:39:16,000: INFO: common: created directory at: artifacts/model_evaluation]


[ModelEvaluationConfig(root_dir='artifacts/model_evaluation', model_name='Ridge', train_data_path='artifacts/data_transformation/train.csv', test_data_path='artifacts/data_transformation/test.csv', model_path='artifacts/model_trainer/models/Ridge/Ridge.pkl', evaluation_save_dir='artifacts/model_evaluation/evaluation', target_column='quality'),
 ModelEvaluationConfig(root_dir='artifacts/model_evaluation', model_name='Lasso', train_data_path='artifacts/data_transformation/train.csv', test_data_path='artifacts/data_transformation/test.csv', model_path='artifacts/model_trainer/models/Lasso/Lasso.pkl', evaluation_save_dir='artifacts/model_evaluation/evaluation', target_column='quality')]

In [20]:
import os
import pandas as pd
from WineQuality import logger
import json
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate_model(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop(columns=[self.config.target_column])
        train_y = train_data[self.config.target_column]

        test_x = test_data.drop(columns=[self.config.target_column])
        test_y = test_data[self.config.target_column]

        logger.info(f"Evaluating {self.config.model_name} model")

        # Load the model
        with open(self.config.model_path, "rb") as f:
            trained_model = pickle.load(f)

        y_pred_train = trained_model.predict(train_x)
        y_pred_test = trained_model.predict(test_x)

        train_metrics = self.calculate_metrics(train_y, y_pred_train)
        test_metrics = self.calculate_metrics(test_y, y_pred_test)

          # Define paths
        train_metric_save_path = os.path.join(self.config.evaluation_save_dir, self.config.model_name, f"{self.config.model_name}_train.json")
        test_metric_save_path = os.path.join(self.config.evaluation_save_dir, self.config.model_name, f"{self.config.model_name}_test.json")

        # Ensure the directories exist (fix: use os.path.dirname)
        os.makedirs(os.path.dirname(train_metric_save_path), exist_ok=True)
        os.makedirs(os.path.dirname(test_metric_save_path), exist_ok=True)

        with open(train_metric_save_path, 'w') as f:
            json.dump(train_metrics, f)

        with open(test_metric_save_path, 'w') as f:
            json.dump(test_metrics, f)

    def calculate_metrics(self, actual, predicted):

        mse = mean_squared_error(actual, predicted)
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)

        return {'MSE':mse,
                'MAE':mae,
                'R2':r2}

        

In [21]:
try:
    config = ConfigurationManager()
    model_evaluation_configs = config.get_model_evaluation_configs()
    for model_evaluation_config in model_evaluation_configs:
        model_evaluatoin = ModelEvaluation(model_evaluation_config)
        model_evaluatoin.evaluate_model()

except Exception as e:
    raise e

[2025-03-22 22:41:38,979: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-22 22:41:38,981: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-22 22:41:38,982: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-22 22:41:38,983: INFO: common: created directory at: artifacts]
[2025-03-22 22:41:38,983: INFO: common: created directory at: artifacts/model_evaluation]
[2025-03-22 22:41:38,986: INFO: 3885688447: Evaluating Ridge model]
[2025-03-22 22:41:38,997: INFO: 3885688447: Evaluating Lasso model]
